## Pydantic Practices
https://docs.pydantic.dev/latest/

### Basic Usage

In [1]:
from pydantic import BaseModel, EmailStr, Field, ValidationError
from typing import Optional

class User(BaseModel):
    name: str = Field(..., min_length=2, max_length=50)
    email: EmailStr
    age: int = Field(..., gt=0, lt=120)
    bio: Optional[str] = Field(default=None, max_length=300)

# Valid input
try:
    user = User(
        name="Alice",
        email="alice@example.com",
        age=30,
        bio="Data scientist who loves clean code."
    )
    print(user)
    print("Serialized to dict:", user.model_dump())
    print("Serialized to JSON:", user.model_dump_json())
except ValidationError as e:
    print("Validation error:", e)

# Invalid input example
try:
    invalid_user = User(
        name="A",
        email="not-an-email",
        age=150
    )
except ValidationError as e:
    print("\nInvalid input caught:")
    print(e)


name='Alice' email='alice@example.com' age=30 bio='Data scientist who loves clean code.'
Serialized to dict: {'name': 'Alice', 'email': 'alice@example.com', 'age': 30, 'bio': 'Data scientist who loves clean code.'}
Serialized to JSON: {"name":"Alice","email":"alice@example.com","age":30,"bio":"Data scientist who loves clean code."}

Invalid input caught:
3 validation errors for User
name
  String should have at least 2 characters [type=string_too_short, input_value='A', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/string_too_short
email
  value is not a valid email address: An email address must have an @-sign. [type=value_error, input_value='not-an-email', input_type=str]
age
  Input should be less than 120 [type=less_than, input_value=150, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/less_than


### Nested Data

In [5]:
from pydantic import BaseModel, Field, StringConstraints
from typing import List, Annotated

class Item(BaseModel):
    name: str = Field(..., min_length=1)
    price: float = Field(..., gt=0, le=100)
    tags: list[Annotated[str, StringConstraints(min_length=1)]] = Field(min_length=1)

class User(BaseModel):
    id: int
    name: str = Field(..., min_length=1)
    email: str

class Order(BaseModel):
    user: User
    items: List[Item]
    total: float
order_data = {
    "user": {
        "id": 123,
        "name": "Alice",
        "email": "alice@example.com"
    },
    "items": [
        {"name": "Keyboard", "price": 49.99, "tags": ["electronics", "accessories"]},
        {"name": "Mouse", "price": 19.99, "tags": ["electronics"]}
    ],
    "total": 69.98
}
try:
    order = Order(**order_data)
    print("Parsed Order:", order)
except Exception as e:
    print("Validation failed:", e)

invalid_data1 = {
    "user": {"id": 1, "name": "", "email": "bob@example.com"},
    "items": [{"name": "Monitor", "price": -100}],
    "total": -100
}
try:
    Order(**invalid_data1)
except ValidationError as e:
    print("Invalid order data caught:")
    print(e)

invalid_data1 = {
    "user": {
        "id": 123,
        "name": "Alice",
        "email": "alice@example.com"
    },
    "items": [
        {"name": "Keyboard", "price": 49.99, "tags": ["electronics", "accessories"]},
        {"name": "Mouse", "price": 19.99, "tags": []}
    ],
    "total": 69.98
}
try:
    Order(**invalid_data1)
except ValidationError as e:
    print("Invalid order data caught:")
    print(e)


Parsed Order: user=User(id=123, name='Alice', email='alice@example.com') items=[Item(name='Keyboard', price=49.99, tags=['electronics', 'accessories']), Item(name='Mouse', price=19.99, tags=['electronics'])] total=69.98
Invalid order data caught:
3 validation errors for Order
user.name
  String should have at least 1 character [type=string_too_short, input_value='', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/string_too_short
items.0.price
  Input should be greater than 0 [type=greater_than, input_value=-100, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/greater_than
items.0.tags
  Field required [type=missing, input_value={'name': 'Monitor', 'price': -100}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
Invalid order data caught:
1 validation error for Order
items.1.tags
  List should have at least 1 item after validation, not 0 [type=too_short, input_value=[]

### Custom Validation

In [3]:
from pydantic import BaseModel, Field, ValidationError, model_validator

class MyModel(BaseModel):
    value1: float = Field(..., gt=0)
    value2: float = Field(..., gt=0)

    @model_validator(mode='after')
    def check_values(cls, model):
        if model.value1 <= 2 * model.value2:
            raise ValueError("value1 must be more than twice value2")
        return model

# ✅ Valid example
try:
    m = MyModel(value1=10, value2=4)
    print("Valid model:", m)
except ValidationError as e:
    print(e)

# ❌ Invalid example
try:
    m = MyModel(value1=6, value2=4)
except ValidationError as e:
    print("Validation failed:\n", e)


Valid model: value1=10.0 value2=4.0
Validation failed:
 1 validation error for MyModel
  Value error, value1 must be more than twice value2 [type=value_error, input_value={'value1': 6, 'value2': 4}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error


### BaseSettings and env

In [4]:
from pydantic import Field, ConfigDict
from pydantic_settings import BaseSettings

class Settings(BaseSettings):
    app_name: str = Field("MyApp", env="APP_NAME", min_length=3)
    debug: bool = Field(False, env="DEBUG")
    port: int = Field(8000, env="PORT")
    db_url: str = Field("hi", env="DB_URL", min_length=5)

    model_config = ConfigDict(
        env_file=".env",
    )

try:
    settings = Settings()
    print("Settings loaded:", settings)
except ValidationError as e:
    print("Settings validation error:", e)


Settings loaded: app_name='test-app' debug=True port=7777 db_url='test-db-url'
